In [5]:
from openai import OpenAI
import gradio as gr
import json

In [6]:
openai = OpenAI()

In [7]:
from typing import Dict, Union

def get_current_temperature(location: str, unit: str) -> Dict[str, Union[float, str]]:
    """
    Tool function for OpenAI Assistant API to get temperature for a location
    
    Args:
        location (str): City name
        unit (str): Temperature unit ("C" or "F")
    
    Returns:
        dict: Contains temperature, unit, location, and timestamp
    """
    # Sample dataset of cities with temperatures in Celsius
    temperature_data = {
        "new york": {"temp": 20, "country": "USA"},
        "london": {"temp": 15, "country": "UK"},
        "tokyo": {"temp": 25, "country": "Japan"},
        "paris": {"temp": 18, "country": "France"},
        "sydney": {"temp": 22, "country": "Australia"}
    }

    def celsius_to_fahrenheit(celsius: float) -> float:
        return (celsius * 9/5) + 32

    try:
        location = location.lower()
        if location not in temperature_data:
            return "I have no data available for this city"

        temp_celsius = temperature_data[location]["temp"]
        temp = celsius_to_fahrenheit(
            temp_celsius) if unit.upper() == "F" else temp_celsius

        return str(temp)

    except Exception as e:
        return {
            "error": str(e),
            "status": "error"
        }

In [8]:
tools = [
  {
    "type": "function",
    "description": "Get the current temperature of specific location",
    "function": {
        "name": "get_current_temperature",
        "description": "Get the current temperature for a specific location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city, e.g. Islamabad, Karachi"
                },
                "unit": {
                    "type": "string",
                    "enum": ["C", "F"],
                    "description": "The temperature unit to use. Infer this from the user's location. C for celcius and F for fahrenheit"
                }
            },
            "required": ["location", "unit"]
        }
    }
  }
]

In [9]:
def chat_with_gpt(message, history):
  messages = history + [{"role": "user", "content": message}]

  # print("Input messages: ", messages)

  completion = openai.chat.completions.create(
      model="gpt-4o",
      messages=messages,
      tools=tools,
  )
  
  # print("Completion: ", completion)
  
  if not completion.choices[0].message.tool_calls:
      return completion.choices[0].message.content
  
  tool_call = completion.choices[0].message.tool_calls[0]
  # print("tool call info: ", tool_call)
  args = json.loads(tool_call.function.arguments)
  result = get_current_temperature(args["location"], args["unit"])

  messages.append(completion.choices[0].message)
  messages.append({                               # append result message
      "role": "tool",
      "tool_call_id": tool_call.id,
      "content": result
  })
  
  print("messages: ", messages)

  # print(messages)

  completion_2 = openai.chat.completions.create(
      model="gpt-4o",
      messages=messages,
      tools=tools,
  )

  return completion_2.choices[0].message.content

In [ ]:
demo = gr.ChatInterface(
    fn=chat_with_gpt,
    type="messages",
    title="Weather Bot",
    flagging_mode="never"
)

demo.launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://722f169d10fe73e346.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


messages:  [{'role': 'user', 'metadata': {'title': None, 'id': None, 'parent_id': None, 'duration': None, 'status': None}, 'content': 'Hello ', 'options': None}, {'role': 'assistant', 'metadata': {'title': None, 'id': None, 'parent_id': None, 'duration': None, 'status': None}, 'content': 'Hi there! How can I assist you today?', 'options': None}, {'role': 'user', 'metadata': {'title': None, 'id': None, 'parent_id': None, 'duration': None, 'status': None}, 'content': 'how are you ?', 'options': None}, {'role': 'assistant', 'metadata': {'title': None, 'id': None, 'parent_id': None, 'duration': None, 'status': None}, 'content': "I'm just a virtual assistant, but I'm here and ready to help you! How can I assist you today?", 'options': None}, {'role': 'user', 'metadata': {'title': None, 'id': None, 'parent_id': None, 'duration': None, 'status': None}, 'content': 'can you provide me the data related to weather', 'options': None}, {'role': 'assistant', 'metadata': {'title': None, 'id': None, '